In [15]:
import sys
sys.path.append("/home/files/uos_plantclassification")
import pandas as pd

exp_path = "/home/files/experiments/mini_plantnet/baseline/EfficientB4/exp_set2"
radius = 25
baseline_metric_path = f"{exp_path}/logs/categorical_metrics_test.csv"
dhc_metric_path = f"{exp_path}/cluster_radius_{radius}/fixed_extractor/logs/categorical_metrics_test.csv"

df1 = pd.read_csv(baseline_metric_path)
df2 = pd.read_csv(dhc_metric_path)

In [16]:
import json
from os.path import join
from os import listdir
import albumentations as A
from albumentations.pytorch import ToTensorV2
from config.path import PATH
from data import get_mini_plantnet, MiniPlantNet
transforms = {
    'train': A.Compose([
        A.Resize(height=380, width=380),
        A.HorizontalFlip(p=0.5),
        A.Normalize(mean=0.0, std=1.0),
        ToTensorV2()]),
    'val': A.Compose([
        A.Resize(height=380, width=380),
        A.Normalize(mean=0.0, std=1.0),
        ToTensorV2()]),
    'test': A.Compose([
        A.Resize(height=380, width=380),
        A.Normalize(mean=0.0, std=1.0),
        ToTensorV2()])
}

dataset = MiniPlantNet(root=PATH["PLANTNET-300K"], split="train", shuffle=False, transform=transforms["test"])

#mini plantnet의 학습데이터 수
label_to_train_samples = {}
name_to_train_samples = {}
for name, label in zip(dataset.name_to_label.keys(), dataset.name_to_label.values()):
    num_samples = len(listdir(join(PATH["PLANTNET-300K"], "images", "train", label)))
    label_to_train_samples[label] = num_samples
    name_to_train_samples[name] = num_samples
train_samples = []
for name in df1.name:
    train_samples.append(name_to_train_samples[name])
df1['train_samples'] = train_samples
df2['train_samples'] = train_samples

# 클러스터 정보
with open(f"{exp_path}/cluster_radius_{radius}/name_clusters.json") as file:
    data = json.load(file)
        
list_of_cluster = list(data.values())
name_to_num_members = {}
name_to_scr = {}
name_to_cluster_id = {}

_id = 0
for cluster in list_of_cluster:
    _id+=1
    num_members = len(cluster)
    samples_of_cluster = 0
    for member in cluster:
        name_to_num_members[member] = num_members
        name_to_cluster_id[member] = _id
        samples_of_cluster += df1[df1.name == member].train_samples.values[0]
    for member in cluster:
        name_to_scr[member] = samples_of_cluster/df1[df1.name == member].train_samples.values[0]
scr = []
num_family_member = []
cluster_id = []
for name in df1.name:
    scr.append(name_to_scr[name])
    num_family_member.append(name_to_num_members[name])
    cluster_id.append(name_to_cluster_id[name])
    
df1['scr'] = scr
df2['scr'] = scr
df1['num_family_member'] = num_family_member
df2['num_family_member'] = num_family_member
df1['cluster_id'] = cluster_id
df2['cluster_id'] = cluster_id

In [17]:
df3 = df2[['name', 'train_samples', 'num_family_member', 'scr', 'cluster_id']]
df3['recall_gap'] = df2['recall'] - df1['recall']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
df = df3.sort_values("recall_gap", ascending=False)

In [19]:
df

,name,train_samples,num_family_member,scr,cluster_id,recall_gap
279,Anthericum ramosum L.,151,2,5.900662,69,0.500000
229,Hypericum pulchrum L.,73,1,1.000000,154,0.300000
390,Anemone virginiana L.,56,1,1.000000,245,0.285714
178,Nephrolepis cordifolia (L.) C.Presl,88,5,15.659091,17,0.250000
140,Acacia longifolia (Andrews) Willd.,104,1,1.000000,95,0.230769
...,...,...,...,...,...,...
463,Peperomia prostrata B.S. Williams,39,1,1.000000,288,-0.333333
159,Mussaenda erythrophylla Schumach. & Thonn.,114,1,1.000000,104,-0.333333
326,Ophrys virescens Philippe,63,1,1.000000,202,-0.333333
417,Garrya elliptica Douglas ex Lindl.,40,1,1.000000,263,-0.400000


In [20]:
df[df.cluster_id==69]

,name,train_samples,num_family_member,scr,cluster_id,recall_gap
279,Anthericum ramosum L.,151,2,5.900662,69,0.500000
92,Anthericum liliago L.,740,2,1.204054,69,-0.043011


In [21]:
df[df.cluster_id==154]

,name,train_samples,num_family_member,scr,cluster_id,recall_gap
229,Hypericum pulchrum L.,73,1,1.0,154,0.3


In [9]:
df[df.cluster_id==40]

,name,train_samples,num_family_member,scr,cluster_id,recall_gap
47,Hypericum hircinum L.,111,5,40.054054,40,0.400000
227,Hypericum calycinum L.,1941,5,2.290572,40,0.114754
328,Hypericum patulum Thunb.,200,5,22.230000,40,0.076923
46,Hypericum androsaemum L.,1359,5,3.271523,40,0.005882
329,Hypericum x hidcoteense Hilling ex Geerinck,835,5,5.324551,40,-0.076190


In [22]:
df[df.cluster_id==171]

,name,train_samples,num_family_member,scr,cluster_id,recall_gap
258,Ophrys lupercalis Devillers & Devillers-Tersch.,40,1,1.0,171,-0.5


In [23]:
df[df.cluster_id==263]

,name,train_samples,num_family_member,scr,cluster_id,recall_gap
417,Garrya elliptica Douglas ex Lindl.,40,1,1.0,263,-0.4


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = df3['scr']
y = df3['recall_gap']

plt.title("acc-gain(%) per scr")
plt.plot(x,y*100,'o')
m, b = np.polyfit(x, y*100, 1)
plt.plot(x, m*x+b)

In [ ]:
df3[df3['num_family_member']==1].recall_gap

In [ ]:
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']==1)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']==2)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']==3)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']==4)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']>4)].recall.mean())

In [ ]:
df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']==3)]

In [ ]:
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<500)*(df1['num_family_member']==1)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<500)*(df1['num_family_member']==2)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<500)*(df1['num_family_member']==3)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<500)*(df1['num_family_member']==4)].recall.mean())
print(df1[(df1['train_samples']>=32)*(df1['train_samples']<500)*(df1['num_family_member']>4)].recall.mean())

In [ ]:
df1[(df1['train_samples']>=32)*(df1['train_samples']<100)*(df1['num_family_member']==1)]

In [ ]:
df3[(df3['train_samples']>=32)*(df3['train_samples']<100)*(df3['num_family_member']==1)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=100)*(df3['train_samples']<200)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=200)*(df3['train_samples']<500)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=500)*(df3['train_samples']<1000)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=1000)*(df3['train_samples']<2000)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=2000)*(df3['train_samples']<5000)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=5000)*(df3['train_samples']<100000)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=32)*(df3['train_samples']<100)*(df3['num_family_member']==1)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=32)*(df3['train_samples']<100)*(df3['num_family_member']==2)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=32)*(df3['train_samples']<100)*(df3['num_family_member']==3)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=32)*(df3['train_samples']<100)*(df3['num_family_member']==4)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=32)*(df3['train_samples']<100)*(df3['num_family_member']>5)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=100)*(df3['train_samples']<200)*(df3['num_family_member']==1)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=100)*(df3['train_samples']<200)*(df3['num_family_member']==2)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=100)*(df3['train_samples']<200)*(df3['num_family_member']==3)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=100)*(df3['train_samples']<200)*(df3['num_family_member']==4)].recall_gap.mean()

In [ ]:
df3[(df3['train_samples']>=100)*(df3['train_samples']<200)*(df3['num_family_member']>5)].recall_gap.mean()

In [ ]:
for i in df2.num_family_member.unique():
    print(i)
    print(df2[(df2['train_samples']>0)*(df2['num_family_member']==i)]['recall'].mean() - df1[(df1['train_samples']<300)*(df1['num_family_member']==i)]['recall'].mean())

In [ ]:
def compare(x,y):
    print(x['recall'].mean() - y['recall'].mean())

In [ ]:
x = df2[(df2['train_samples']>0)*(df2['train_samples']<=100)]
y = df1[(df2['train_samples']>0)*(df2['train_samples']<=100)]
compare(x,y)
x = df2[(df2['train_samples']>100)*(df2['train_samples']<=300)]
y = df1[(df2['train_samples']>100)*(df2['train_samples']<=300)]
compare(x,y)
x = df2[(df2['train_samples']>300)*(df2['train_samples']<=500)]
y = df1[(df2['train_samples']>300)*(df2['train_samples']<=500)]
compare(x,y)
x = df2[(df2['train_samples']>500)*(df2['train_samples']<=1000)]
y = df1[(df2['train_samples']>500)*(df2['train_samples']<=1000)]
compare(x,y)
x = df2[(df2['train_samples']>1000)*(df2['train_samples']<=2000)]
y = df1[(df2['train_samples']>1000)*(df2['train_samples']<=2000)]
compare(x,y)

x = df2[(df2['train_samples']>2000)*(df2['train_samples']<=10000)]
y = df1[(df2['train_samples']>2000)*(df2['train_samples']<=10000)]
compare(x,y)

In [ ]:
x = df2[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==1)]
y = df1[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==1)]
compare(x,y)
x = df2[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==2)]
y = df1[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==2)]
compare(x,y)
x = df2[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==3)]
y = df1[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==3)]
compare(x,y)
x = df2[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==4)]
y = df1[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==4)]
compare(x,y)
x = df2[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==5)]
y = df1[(df2['train_samples']>300)*(df2['train_samples']<=500)*(df2['num_family_member']==5)]
compare(x,y)

In [ ]:
df3 = df2[['name', 'train_samples', 'num_family_member']]
df3['recall_gap'] = df2['recall'] - df1['recall']

In [ ]:
df3[(df3['train_samples']>300)*(df3['train_samples']<=500)]

In [ ]:
df3[(df3['train_samples']<500)][:50]

In [ ]:
import plotly.express as px
fig = px.scatter_3d(df3, x='train_samples', y='num_family_member', z='recall_gap')
fig.update_traces(marker_size = 1)


In [ ]:
from matplotlib import pyplot as plt



X = df3['train_samples']
Y = df3['num_family_member']
Z = df3['recall_gap']

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(X,Y,Z)


In [ ]:
#df3[df3['train_samples']<100].recall_gap.mean()
df3[df3['train_samples']>100].recall_gap.mean()

In [ ]:
df3[df3['train_samples']<100].recall_gap.mean()

In [ ]:
df2[(df2['train_samples']<300)*(df2['num_family_member']==3)]['recall'].mean() - df1[(df1['train_samples']<300)*(df1['num_family_member']==3)]['recall'].mean()

In [ ]:
df2[(df2['train_samples']<300)*(df2['num_family_member']==5)]['recall'].mean() - df1[(df1['train_samples']<300)*(df1['num_family_member']==5)]['recall'].mean()

In [ ]:
df2[(df2['train_samples']<300)*(df2['num_family_member']==6)]['recall'].mean() - df1[(df1['train_samples']<300)*(df1['num_family_member']==6)]['recall'].mean()

In [ ]:
df2[(df2['train_samples']<300)*(df2['num_family_member']==6)]['recall'].mean() - df1[(df1['train_samples']<300)*(df1['num_family_member']==6)]['recall'].mean()

In [ ]:
(df2.recall-df1.recall).mean()

In [ ]:
df2.recall.mean() - df1.recall.mean()

In [ ]:
df2['recall'].mean()

In [ ]:
print(df1[df1.samples_per_class<100]['recall'].mean())
print(df2[df2.samples_per_class<100]['recall'].mean())

In [ ]:
print(df1[(500<df1.samples_per_class)*(df1.samples_per_class<2000)]['recall'].mean())
print(df2[(500<df1.samples_per_class)*(df1.samples_per_class<2000)]['recall'].mean())

In [ ]:
for x in range(10):
    if 3<x<8:
        print(x)